# Import packages

In [ ]:
from numpy import empty
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import csv
from pandas import read_csv
from datetime import date

# Create csv file to store results

In [ ]:
# defining file name
today = date.today()
fileName = 'scraping_{}.csv'.format(today.strftime("%b-%d-%Y"))

# create csv file
csv_file = open(fileName, 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file, delimiter = ';')

# write header names
writer.writerow(['url', 'title', 'views', 'likes', 'comments', 'comments_likes'])

# dict for put the info
youtube_info = {}

# Import YouTube links (in 'file_links' put the file path with the links)

In [ ]:
# read youtube links
file_links ='needed_files\youtube_links.txt' 
youtube_links = [l[0] for l in read_csv(file_links, header=None).values]

# Scraping

In [ ]:
# scraping initializing
for url in youtube_links:
    # open chrome
    file_chromedriver = 'needed_files\chromedriver.exe'
    driver = webdriver.Chrome(service=Service(file_chromedriver))
    driver.get(url)
    driver.maximize_window()
    time.sleep(10)

    # collect info
    title = driver.find_elements(By.XPATH,'//*[@id="container"]/h1/yt-formatted-string')[0].text
    views = driver.find_elements(By.XPATH,'//*[@id="count"]/ytd-video-view-count-renderer/span[1]')[0].text
    likes = driver.find_elements(By.XPATH, '//*[@id="text"]')[2].text

    # number of comments
    driver.execute_script('window.scrollTo(0,390);')
    time.sleep(2)  # time for loading the video
    n_comments = int(driver.find_elements(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[1]')[0].text.replace('.',''))

    # scroll down to load comments
    total_comments = False
    i = 390; k = 4000
    while not total_comments:
        for j in range(i, k, 500):
            driver.execute_script('window.scrollTo({},{});'.format(i,j))
            time.sleep(2)

        # Extract comments info
        comments_info = driver.find_elements(By.CSS_SELECTOR,'#contents #comment')

        # Extract comments
        comments = [c.text for c in driver.find_elements(By.XPATH, '//*[@id="content-text"]')]
        commentsLikes = [upvote.text for upvote in driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')]

        # Find comments with replies
        n_replie = 0
        replies = driver.find_elements(By.XPATH,'//*[@id="replies"]')
        for r in replies:
            if r.text != '':
                try:
                    n_replie = n_replie + int(r.text.split(' ')[1]) # if there is an error it is because it is a single answer. There is no "1 answer"
                except:
                    n_replie = n_replie + 1

        # percentage of comments obtained
        commentPercentObtained = (len(comments)+n_replie)/n_comments
        
        if commentPercentObtained >= 0.9:
            total_comments = True 
        else:
            i = j; k = 2 * j

    # Close the googledriver
    driver.close()

    # fill the dict
    for i in range(0,len(comments),1):
        youtube_info['url'] = url
        youtube_info['title'] = title
        youtube_info['views'] = views
        youtube_info['likes'] = likes
        youtube_info['comments'] = comments[i]
        youtube_info['comments_likes'] = commentsLikes[i]

        # write results in csv file
        writer.writerow(youtube_info.values())

csv_file.close()